In [116]:
import pandas as pd
import datetime
from math import radians, cos, sin, asin, sqrt
import numpy as np

In [164]:
def parse_timestamp(time_in_secs):    
    return datetime.datetime.fromtimestamp(int(time_in_secs) / 1e6)

def df_dist(row):
    return haversine(row['Lon'], row['Lat'], row['Lon_prev'], row['Lat_prev'])

def df_speed(row):
    if not row['Time deltas'].total_seconds():
        return 0
    m_s = row['Traveled dist'] * 1000 / row['Time deltas'].total_seconds()
    km_h = m_s * 3.6
    return km_h

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [102]:
# Columns description
# Timestamp micro since 1970 01 01 00:00:00 GMT'
# Line ID
# Direction
# Journey Pattern ID
# Time Frame (The start date of the production time table - in Dublin the production time table starts at 6am and ends at 3am)
# Vehicle Journey ID (A given run on the journey pattern)
# Operator (Bus operator, not the driver)
# Congestion [0=no,1=yes]
# Lon WGS84
# Lat WGS84
# Delay (seconds, negative if bus is ahead of schedule)
# Block ID (a section ID of the journey pattern)
# Vehicle ID
# Stop ID
# At Stop [0=no,1=yes]

categorical_columns = ['Line ID', 'Direction', 'Journey Pattern ID', 'Vehicle Journey ID', 'Operator', 'Congestion', 'Block ID', 'Vehicle ID', 'Stop ID', 'At Stop']
df = pd.read_csv('sir140113-200113/siri.20130114.csv.gz',
                 names=['Timestamp', 'Line ID', 'Direction', 'Journey Pattern ID', 'Time Frame', 'Vehicle Journey ID', 'Operator', 'Congestion', 'Lon', 'Lat', 'Delay', 'Block ID', 'Vehicle ID', 'Stop ID', 'At Stop'],
                 parse_dates=[0], date_parser=parse_timestamp,
                 dtype={col_name: 'category' for col_name in categorical_columns})

In [103]:
df.columns

Index(['Timestamp', 'Line ID', 'Direction', 'Journey Pattern ID', 'Time Frame',
       'Vehicle Journey ID', 'Operator', 'Congestion', 'Lon', 'Lat', 'Delay',
       'Block ID', 'Vehicle ID', 'Stop ID', 'At Stop'],
      dtype='object')

In [104]:
df.dtypes

Timestamp             datetime64[ns]
Line ID                     category
Direction                   category
Journey Pattern ID          category
Time Frame                    object
Vehicle Journey ID          category
Operator                    category
Congestion                  category
Lon                          float64
Lat                          float64
Delay                          int64
Block ID                    category
Vehicle ID                  category
Stop ID                     category
At Stop                     category
dtype: object

In [140]:
for c in categorical_columns:
    print(c)
    print(df[c].unique())
    print('-----')

Line ID
[66, 40, 27, 39, 46, ..., 104, 53, 51, 118, NaN]
Length: 66
Categories (65, object): [66, 40, 27, 39, ..., 104, 53, 51, 118]
-----
Direction
[0]
Categories (1, object): [0]
-----
Journey Pattern ID
[00660001, 040D0001, 077A1001, 039A0001, 046A0001, ..., 00111002, 00371002, 014C0001, 01851008, 00331005]
Length: 444
Categories (443, object): [00660001, 040D0001, 077A1001, 039A0001, ..., 00371002, 014C0001, 01851008, 00331005]
-----
Vehicle Journey ID
[14217, 14403, 14130, 14729, 16565, ..., 5924, 7291, 6674, 4288, 5340]
Length: 7503
Categories (7503, object): [14217, 14403, 14130, 14729, ..., 7291, 6674, 4288, 5340]
-----
Operator
[PO, HN, RD, D1, CF, SL, CD, D2]
Categories (8, object): [PO, HN, RD, D1, CF, SL, CD, D2]
-----
Congestion
[0, 1]
Categories (2, object): [0, 1]
-----
Block ID
[66006, 40204, 27009, 39015, 46007, ..., 9012, 145001, 46030, 83015, 145070]
Length: 868
Categories (868, object): [66006, 40204, 27009, 39015, ..., 145001, 46030, 83015, 145070]
-----
Vehicle ID

In [105]:
df.head()

,Timestamp,Line ID,Direction,Journey Pattern ID,Time Frame,Vehicle Journey ID,Operator,Congestion,Lon,Lat,Delay,Block ID,Vehicle ID,Stop ID,At Stop
0,2013-01-14 04:00:01,66,0,00660001,2013-01-13,14217,PO,0,-6.569710,53.380451,222,66006,40001,3968,0
1,2013-01-14 04:00:01,40,0,040D0001,2013-01-13,14403,HN,0,-6.373083,53.410049,338,40204,38067,6005,0
2,2013-01-14 04:00:01,27,0,077A1001,2013-01-13,14130,RD,0,-6.259118,53.345650,-534,27009,33254,1358,0
3,2013-01-14 04:00:01,39,0,039A0001,2013-01-13,14729,PO,0,-6.274983,53.350784,-125,39015,33557,7160,0
4,2013-01-14 04:00:01,46,0,046A0001,2013-01-13,16565,D1,0,-6.230600,53.317665,-988,46007,33532,2032,0


In [106]:
df.describe()

,Lon,Lat,Delay
count,1.725539e+06,1.725539e+06,1.725539e+06
mean,-6.271743e+00,5.334475e+01,-7.030397e+01
std,8.367796e-02,5.517641e-02,4.611429e+02
min,-6.615016e+00,5.307040e+01,-4.698000e+03
25%,-6.307931e+00,5.331962e+01,-2.580000e+02
50%,-6.261075e+00,5.334644e+01,0.000000e+00
75%,-6.231783e+00,5.337473e+01,6.100000e+01
max,-6.053017e+00,5.360652e+01,3.136200e+04


In [107]:
# Ugly way to make coordinates start at 0. http://spatialreference.org/ref/epsg/wgs-84/
df['Lat'] += 180
df['Lon'] += 90

In [108]:
df.describe()

,Lon,Lat,Delay
count,1.725539e+06,1.725539e+06,1.725539e+06
mean,8.372826e+01,2.333448e+02,-7.030397e+01
std,8.367796e-02,5.517641e-02,4.611429e+02
min,8.338498e+01,2.330704e+02,-4.698000e+03
25%,8.369207e+01,2.333196e+02,-2.580000e+02
50%,8.373892e+01,2.333464e+02,0.000000e+00
75%,8.376822e+01,2.333747e+02,6.100000e+01
max,8.394698e+01,2.336065e+02,3.136200e+04


In [109]:
lat_min = df['Lat'].min()
lat_range = df['Lat'].max() - lat_min
lon_min = df['Lon'].min()
lon_range = df['Lon'].max() - lon_min

In [112]:
# Get coordinate step for given number of cells.
num_cells = 150
lat_step = lat_range / num_cells
lon_step = lon_range / num_cells
print(lat_step)
print('lat step: {:.2f}km'.format(haversine(0, 0, lat_step, 0)))
print('lon step: {:.2f}km'.format(haversine(0, 0, lon_step, 0)))

0.00357412
lat step: 0.40km
lon step: 0.42km


In [ ]:
%%time
X = []
y = []
for lat_idx in range(25):
    print(lat_idx)
    lat_start, lat_stop = lat_min + lat_idx * lat_step, lat_min + (lat_idx + 1) * lat_step
    df_lat_slice = df[(lat_start <= df['Lat']) & (df['Lat'] < lat_stop)]
    for lon_idx in range(num_cells):
        lon_start, lon_stop = lon_min + lon_idx * lon_step, lon_min + (lon_idx + 1) * lon_step
        cell_data = df_lat_slice[(lon_start <= df_lat_slice['Lon']) & (df_lat_slice['Lon'] < lon_stop)]
        cell_value_counts = cell_data['At Stop'].value_counts()
        if True or cell_value_counts[1] > 0:
            line_ids = cell_data['Line ID'].unique()
            for line_id in line_ids:
                line_id_df = cell_data[cell_data['Line ID'] == line_id]
                journey_ids = line_id_df['Vehicle Journey ID'].unique()
                value_counts = np.zeros(2)
                line_hist = None
                for journey_id in journey_ids:
                    journey_df = line_id_df[line_id_df['Vehicle Journey ID'] == journey_id]
                    if len(journey_df) < 2:
                        continue
                    
                    journey_df = journey_df.sort_values('Timestamp')
                    journey_df = journey_df.join(journey_df[['Timestamp', 'Lon', 'Lat']].shift(), rsuffix='_prev')
                    journey_df['Time deltas'] = journey_df['Timestamp'] - journey_df['Timestamp_prev']
                    journey_df['Traveled dist'] = journey_df[['Lon', 'Lat', 'Lon_prev', 'Lat_prev']].apply(df_dist, axis=1)
                    journey_df['Speed'] = journey_df.apply(df_speed, axis=1)
                                        
                    journey_value_counts = journey_df['At Stop'].value_counts(sort=False)
                    features = np.histogram(journey_df['Speed'], bins=20, range=(0, 60))[0]
                    if line_hist is None:
                        line_hist = features
                    else:
                        line_hist += features
                    value_counts += journey_value_counts
                if line_hist is not None:
                    X.append(line_hist / np.sum(line_hist))
                    is_stop = value_counts[1] > value_counts[0]
                    y.append(is_stop)

0


/home/sokolov/miniconda3/envs/bus-stops/lib/python3.5/site-packages/numpy/lib/function_base.py:583: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= mn)
/home/sokolov/miniconda3/envs/bus-stops/lib/python3.5/site-packages/numpy/lib/function_base.py:584: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= mx)


1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [226]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support

# print(X)
# print(X / np.linalg.norm(X, axis=1).reshape(-1, 1))
X = np.array(X)
y = np.array(y)
print(np.unique(y, return_counts=True))

clf = RandomForestClassifier()
precs, recs, f1s, sups = [], [], [], []
kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
for train_idx, val_idx in kf.split(X, y):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    clf.fit(X_train, y_train)
    pr, rec, f1, sup = precision_recall_fscore_support(y_val, clf.predict(X_val))
    precs.append(pr)
    recs.append(rec)
    f1s.append(f1)
    sups.append(sup)

print("Precision: %0.2f (+/- %0.2f)" % (np.mean(precs), np.std(precs) * 2))
print("Recall: %0.2f (+/- %0.2f)" % (np.mean(recs), np.std(recs) * 2))
print("F1s: %0.2f (+/- %0.2f)" % (np.mean(precs), np.std(f1s) * 2))
print("Support: %0.2f (+/- %0.2f)" % (np.mean(sups), np.std(sups) * 2))

(array([False,  True], dtype=bool), array([21,  5]))
Precision: 0.39 (+/- 0.79)
Recall: 0.45 (+/- 0.92)
F1s: 0.39 (+/- 0.84)
Support: 2.60 (+/- 3.25)


/home/sokolov/miniconda3/envs/bus-stops/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sokolov/miniconda3/envs/bus-stops/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sokolov/miniconda3/envs/bus-stops/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
